### LLM Agent with Function Calling

The agent monitors a mailbox—in this case, Gmail—and alerts you about emails that need attention and prompt responses. 


In [ ]:
%pip install -q openai python-dotenv google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib bs4

In [ ]:
import sys
from openai import AzureOpenAI
import json
import os
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
from gmail_functions import get_gmail_service, \
    get_current_date, \
    list_messages, \
    classify_and_summarize_email, \
    send_email
current_directory = os.getcwd()
sys.path.append(current_directory)

In [ ]:
# Configure Azure Open AI
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
# Details about supported API versions: https://learn.microsoft.com/en-us/azure/ai-services/openai/reference
os.environ["OPENAI_API_VERSION"] = "2024-03-01-preview"
os.environ["OPENAI_API_TYPE"] = "azure"
# set gmail credentials
sec_profile = get_gmail_service()

#### Note that in order to run this code, you need to have a Gmail account and enable the Gmail API.

Here https://developers.google.com/gmail/api/quickstart/python you can find a guide on how to enable the Gmail API.

In [ ]:
# Authenticate against Gmail API
sec_profile = get_gmail_service()

In [ ]:
# initialize the model
llm = AzureOpenAI()

# Prepare prompt
messages = [
    {"role": "system", "content": """You are a professional email assistant. Carefully analyze the input emails according to the provided instructions.
                                Please review all my emails received today only and perform the following tasks:
                                1.	For each email, determine if it belongs to one of the following categories:
                                    a. CONCERN: The sender is concerned about something.
                                    b. QUESTION: The sender is inquiring about a particular issue.
                                    c. RESPONSE IS AWAITED: The sender is awaiting a response.
                                    d. NEGATIVE SENTIMENT: The sender is expressing negative sentiment.
	                            2.	Send me an email with the following content in the body for each relevant email:
	                                    •	A summary of the original email.
	                                    •	One or more of the categories listed above.
	                                    •	The sender of the original email.
                                For classification, summarization, and sentiment analysis, use ONLY the meaningful textual content of the email.
                                If an email does not fall into any of the categories (CONCERN, QUESTION, RESPONSE AWAITED, or NEGATIVE SENTIMENT), do not include it in the output.
                                DO NOT classify the same email into more than one category.
                                DO NOT make assumptions about the sender’s intentions; base your reasoning solely on the facts presented in the email.
                                DO NOT process emails with the subject: “Important. Attention is required!”
                                DO NOT process emails that contain promotional offers or any form of advertising.
     """},
]

In [ ]:
tools = [
    {"type": "function",
     "function": {
             'name': 'get_current_date',
             'description': 'Returns the current date in the format YYYY/MM/DD',
     }
     },
    {
        "type": "function",
        "function": {
            'name': 'classify_and_summarize_email',
            'description': 'Classifies the email to one or more of the following categories: CONCERN, QUESTION, RESPONSE IS AWAITED, NEGATIVE SENTIMENT. Summarizes the email and returns:  category, summary and the sender email.',
            'parameters': {'type': 'object',
                           'properties': {'category': {'description': 'email category. Category value must be one of : CONCERN, QUESTION, RESPONSE IS AWAITED, MEGATIVE SENTIMENT', 'type': 'string'},
                                          'summary': {'description': 'email summary', 'type': 'string'},
                                          'sender_email': {'description': 'sender email', 'type': 'string'}
                                          },
                           'required': ['category', 'summary', 'sender_email']}}
    },
    {"type": "function",
     "function": {
             'name': 'NER',
             'description': 'Extract all named entities from the email subject and body. NER stands for Named Entity Recognition',
             'parameters': {'type': 'object',
                            'properties': {'person': {'description': 'person name', 'type': 'string'},
                                           'company': {'description': 'company name', 'type': 'string'},
                                           'job_title': {'description': 'person job title', 'type': 'string'}
                                           },
                            'required': ['person', 'company', 'job_title']}}
     },
    {
        "type": "function",
        "function": {
                "name": "list_messages",
                "description": "List all messages in the user's mailbox that match the query and output as JSON and returns for every message the subject and body.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "service": {
                            "type": "object",
                            "description": "Authorized Gmail API service instance."
                        },
                        "user_id": {
                            "type": "string",
                            "description": "User's email address. The special value 'me' can be used to indicate the authenticated user."
                        },
                        "max_results": {
                            "type": "integer",
                            "description": "Maximum number of messages to return."
                        },
                        "query": {
                            "type": "string",
                            "description": "String used to filter the messages listed."
                        }
                    },
                    "required": ["service", "user_id", "max_results"]
                }
        }
    },
    {
        "type": "function",
        "function": {
                "name": "send_email",
                "description": "Send an email .",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "service": {
                            "type": "object",
                            "description": "Authorized Gmail API service instance."
                        },
                        "user_id": {
                            "type": "string",
                            "description": "User's email address. The special value 'me' can be used to indicate the authenticated user."
                        },
                        "message": {
                            "type": "string",
                            "description": "The body of the email being sent."
                        }
                    },
                    "required": ["service", "user_id", "email_body"]
                }
        }
    },
]

In [ ]:
response = llm.chat.completions.create(
    model="gpt-4-turbo",
    temperature=0,
    messages=messages,
    tools=tools,
)
response

In [ ]:

supported_functions = {"get_current_date": get_current_date,
                       "list_messages": list_messages,
                       "classify_and_summarize_email": classify_and_summarize_email,
                       "send_email": send_email,
                       }

In [ ]:
# make it generic with multiple calls
while (response.choices[0].finish_reason == 'tool_calls'):

    #append model response to the "messages"
    messages.append(response.choices[0].message)

    for tool in response.choices[0].message.tool_calls:
        if tool.function.name in supported_functions:
            function_arguments = json.loads(tool.function.arguments)
            # execute the function
            function_response = supported_functions[tool.function.name](
                sec_profile, **function_arguments)

            #append function output to the "messages"
            messages.append(
                {
                    "tool_call_id": tool.id,
                    "role": "tool",
                    "name": tool.function.name,
                    "content": function_response,
                }
            )
    response = llm.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0,
        messages=messages,
        tools=tools,
    )
    print (response)
# finish_reason='stop'
print(response.choices[0].message)

In [ ]:
messages